<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Add-CWL-python-source-to-classpath" data-toc-modified-id="Add-CWL-python-source-to-classpath-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Add CWL python source to classpath</a></span></li><li><span><a href="#Search-trajectory-files-and-create-output-path" data-toc-modified-id="Search-trajectory-files-and-create-output-path-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Search trajectory files and create output path</a></span></li><li><span><a href="#Plot-experiment-trajectories" data-toc-modified-id="Plot-experiment-trajectories-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Plot experiment trajectories</a></span></li><li><span><a href="#Processing-Experiments" data-toc-modified-id="Processing-Experiments-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Processing Experiments</a></span></li><li><span><a href="#Interactive-plot-of-density-images" data-toc-modified-id="Interactive-plot-of-density-images-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Interactive plot of density images</a></span></li><li><span><a href="#Aggregate-unfiltered-density-distributions" data-toc-modified-id="Aggregate-unfiltered-density-distributions-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Aggregate unfiltered density distributions</a></span></li><li><span><a href="#Filter-duplicated-images" data-toc-modified-id="Filter-duplicated-images-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Filter duplicated images</a></span></li><li><span><a href="#Aggregate-filtered-density-distributions" data-toc-modified-id="Aggregate-filtered-density-distributions-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Aggregate filtered density distributions</a></span></li><li><span><a href="#Plot-density-maps-to-file" data-toc-modified-id="Plot-density-maps-to-file-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Plot density maps to file</a></span></li></ul></div>

In [2]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as matplot
import itertools
import math
import sys

from tqdm import tqdm_notebook, tqdm
from os import listdir, mkdir
from os.path import join, isdir
from ipywidgets import *
from multiprocessing import Pool, Lock

%load_ext autoreload
%autoreload 1
%aimport preprocessing, plotting

%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Add CWL python source to classpath

In [7]:
# cwl library imports

module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from utils.reader import get_all_trajectory_files, read_trajectory_file


# Search trajectory files and create output path

In [8]:
# paramter
input_path = 'I:/projects/vadere/VadereModelTests/TestCWL/output'
output_path = 'juelich-vadere/output_01/'
experiments = get_all_trajectory_files(input_path)

if not isdir(output_path):
    mkdir(output_path)

# Plot experiment trajectories

In [ ]:
# helper functions for plotting

def toInch(pixel, dpi=92):
    return pixel / dpi


def plot_experiment(frame, ax):
    # map p-ids to targets
    for pId, group in frame.groupby('pedestrianId'):
        data = group.loc[(group['x'] > 14.5) & (
            group['x'] < 24.9) & (group['y'] < 8.7)]
        ax.plot(data['x'], data['y'], zorder=1)

    rect = matplot.Rectangle(
        (18.49, 6.69), 2.4, 1, edgecolor='r', alpha=1, fill=False, zorder=2, linewidth=2)

    ax.add_patch(rect)
    ax.grid()

In [ ]:
# plot all scenario trajectories to file
%% capture
plots = len(experiments)
rows = math.ceil(plots / 2.0)
fig, ax = matplot.subplots(nrows=rows, ncols=2, figsize=(
    toInch(1000), toInch(rows * 300)), dpi=92)

ax = np.array(ax).flatten()

for idx in range(plots):
    experiment = experiments[idx]
    directory = os.path.dirname(experiment)
    name = os.path.basename(directory).split('_')[0]

    ax[idx].set_title(name)

    frame = read_trajectory_file(experiment)
    plot_experiment(frame, ax[idx])

fig.subplots_adjust(hspace=0.25)
fig.savefig('scenario-plots.pdf', transparent=True)

# Processing Experiments

In [ ]:
# processing experiments

results = []
for experiment in tqdm_notebook(experiments, desc='process files'):
    directory = os.path.dirname(experiment)
    name = os.path.basename(directory)

    if os.path.exists(join(output_path, name + '.csv')):
        print('skipping', name)
        continue

    frame = read_trajectory_file(experiment)
    pId2Target = {r['pedestrianId']: r['sourceId'] for i, r in pd.read_csv(
        join(directory, 'sources.csv'), sep=' ').iterrows()}
    context = dict({
        'name': name,
        'targets': [0, 1, 2],
        'area':  [18.50, 6.7, 2.4, 1],
        'exact': True,
        'sigma': 0.7,
        'resolution': 0.1,
        'pedestrian_radius': 0.195,
        'gauss_bounds': 2,
        'pId2Target': pId2Target,
        'processors': [
            preprocessinge.process_densities,
            preprocessinge.process_pedestrians,
            preprocessinge.process_percentiles
        ],
        'number_of_cores': 3,
        'skip': 1
    })
    context = preprocessinge.process_experiment(frame, context)
    results.append(context)

    densities = context.get('densities')
    with open(join(output_path, name + '.csv'), mode='w') as file:
        for density in densities:
            file.write(
                ';'.join(map(lambda r: "{:.10f}".format(r), density)) + '\n')

        file.flush()

# Interactive plot of density images
<font color='red' size='5'><b>Important note:</b><br/>This requires that the result of the processed files (cell above) is still available</font>

In [ ]:
# interactive density maps
%matplotlib notebook

def extrac_xy(pedestrians, x0, y0, resolution):
    x = []
    y = []

    for _, ped in pedestrians.iterrows():
        x.append(int(abs(ped['x'] - x0) * (1 / resolution)))
        y.append(int(abs(ped['y'] - y0) * (1 / resolution)))

    return x, y

for result in results:
    name = result.get('name')
    densities = result.get('densities')
    area = result.get('area')
    peds = result.get('pedestrians')
    size = result.get('size')
    res = result.get('resolution')
    t = len(result.get('targets'))

    start = 0

    first = densities[start][:-t].reshape(size)

    fig, ax = matplot.subplots()

    cbar = None

    ax.set_xticks(np.linspace(0, len(first[0]) - 1, 13))
    ax.set_xticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2,
                        1.4, 1.6, 1.8, 2.0, 2.2, 2.4])

    ax.set_yticks(np.linspace(0, len(first) - 1, 6))
    ax.set_yticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0])

    def update(frame):
        global cbar
        f = densities[frame][:-t].reshape(size)

        ax.clear()
        if cbar is not None:
            cbar.remove()

        img = ax.imshow(f, interpolation='Nearest')
        cbar = fig.colorbar(img)

        x, y = extrac_xy(peds[frame], area[0], area[1], res)

        ax.plot(x, y, '.r', markersize=10)

        ax.set_xticks(np.linspace(0, len(first[0]) - 1, 13))
        ax.set_xticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0,
                            1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])

        ax.set_yticks(np.linspace(0, len(first) - 1, 6))
        ax.set_yticklabels([0, 0.2, 0.4, 0.6, 0.8, 1.0])

    interact(update, frame=widgets.IntSlider(
        min=0, max=len(densities)-1, step=1, value=start))

    break

# Aggregate unfiltered density distributions

In [ ]:
# aggregate distributions

# output_p = 'juelich-vadere/output_01_save/'
outputs = list(filter(lambda o: 'filtered' not in o, os.listdir(output_path)))
dfs = []
for output in outputs:
    dfs.append(pd.read_csv(os.path.join(
        output_path, output), sep=';', header=None))

frame = pd.concat([df.iloc[:, -3:] for df in dfs], sort=False)
display(frame.groupby([230, 231, 232], as_index=False).size().to_frame(
    'frames').reset_index())

# Filter duplicated images

In [ ]:
# filter consecutive duplicates

outputs = list(filter(lambda o: 'filtered' not in o, os.listdir(output_path)))

for output in tqdm_notebook(outputs, desc="filter density maps"):
    last = None
    skipped = 0
    written = 0
    with open(os.path.join(output_path, output), "r") as full:
        with open(os.path.join(output_path, output[:-4] + '_filtered.csv'), "w") as filtered:
            for line in full:
                if line != last:
                    filtered.write(line)
                    last = line
                    written = written + 1
                    continue

                skipped = skipped + 1
    print(written, 'lines written,', skipped, 'lines skipped')

# Aggregate filtered density distributions

In [ ]:
# aggregate distributions

# output_p = 'juelich-vadere/output_01_save/'
outputs = list(filter(lambda o: 'filtered' in o, os.listdir(output_path)))
dfs = []
for output in outputs:
    dfs.append(pd.read_csv(os.path.join(
        output_path, output), sep=';', header=None))

frame = pd.concat([df.iloc[:, -3:] for df in dfs], sort=False)
display(frame.groupby([230, 231, 232], as_index=False).size().to_frame(
    'frames').reset_index())

# Plot density maps to file

In [12]:
# save density maps

image_directory = "juelich-vadere/images_01_1/"

if not os.path.isdir(image_directory):
    os.mkdir(image_directory)

def filter_chunk(chunk):
    return chunk.loc[(chunk[231] == 0.5) & (chunk[232] == 0.5)]
    
outputs = list(filter(lambda o: 'filtered' in o, os.listdir(output_path)))
for output in tqdm_notebook(outputs):
    plotting.processFile(output, output_path, image_directory, _filter=filter_chunk)

Path juelich-vadere/images_01_1/KO-240-050-240_2019-03-28_09-53-45.754_filtered already exists


Done KO-240-050-240_2019-03-28_09-53-45.754_filtered.csv


Done KO-240-050-240_2019-03-28_10-15-22.564_filtered.csv


Done KO-240-050-240_2019-03-28_10-23-27.825_filtered.csv


Done KO-240-060-240_2019-03-28_09-54-35.343_filtered.csv


...

Done KO-240-100-240_2019-03-28_09-56-16.552_filtered.csv


Done KO-240-100-240_2019-03-28_10-18-05.949_filtered.csv


Done KO-240-100-240_2019-03-28_10-26-05.47_filtered.csv


Done KO-240-120-240_2019-03-28_09-57-08.580_filtered.csv


KeyboardInterrupt: 